# Рекомендация тарифов


**Заказчик** — Оператор мобильной связи «Мегалайн»

**Цель исследования** 

* Проанализировать поведение клиентов 


**Входные данные**  — данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»).


**Результаты исследования** — Получение модели с максимально большим значением *accuracy* с долей правильных ответов по крайней мере до 0.75.

## Откройте и изучите файл

Откройте файл с данными и изучите его. Путь к файлу: `/datasets/users_behavior.csv`

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score


In [2]:
df = pd.read_csv('datasets/users_behavior.csv')

# посмотрю что выходит, возможно нужно загружать с разделителем
df

#все ок

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


**Описание таблицы**  
<a id='info_DB'></a>

* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [3]:
print('Информация по таблице:', '\n')
df.info()
print()
print('Количество пропусков:', '\n')
print(df.isna().sum(), '\n', '\n')
print('Процент пропусков:', '\n')
print(df.isna().mean()*100, '\n')

display(df.describe())
print()
print('Число явных дубликатов:', df.duplicated().sum(), '\n')

Информация по таблице: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB

Количество пропусков: 

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64 
 

Процент пропусков: 

calls       0.0
minutes     0.0
messages    0.0
mb_used     0.0
is_ultra    0.0
dtype: float64 



,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000



Число явных дубликатов: 0 



Пропусков и дублей нет, названия нормальные

Переведём: `calls` и `messages` в int

In [4]:
df['calls'] = df['calls'].astype('int')
df['messages'] = df['messages'].astype('int')

print('Информация по таблице:', '\n')
df.info()

df.head()

Информация по таблице: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int32  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int32  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int32(2), int64(1)
memory usage: 100.6 KB


,calls,minutes,messages,mb_used,is_ultra
0,40,311.90,83,19915.42,0
1,85,516.75,56,22696.96,0
2,77,467.66,86,21060.45,0
3,106,745.53,81,8437.39,1
4,66,418.74,1,14502.75,0


## Разбейте данные на выборки

Разделите исходные данные на обучающую, валидационную и тестовую выборки.

In [5]:
# извлеку признаки и целевой признак

features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

#Разбиваю исходные данные в соотношении 3:1:1. 
#Вначале на обучающую 60% и валид+тестовая 40%
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target,  test_size=0.40, random_state=12345)

# валид+тестовая делю пополам 
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid,  test_size=0.50, random_state=12345)


In [6]:
print('Обучающая база:')
print('Колличество - признак:', len(features_train))
print('Колличество - целевой признак:', len(target_train), '\n')

print('Валидационная база:')
print('Колличество - признак:', len(features_valid))
print('Колличество - целевой признак:', len(target_valid), '\n')

print('Тестовая база:')
print('Колличество - признак:', len(features_test))
print('Колличество - целевой признак:', len(target_test), '\n')

Обучающая база:
Колличество - признак: 1928
Колличество - целевой признак: 1928 

Валидационная база:
Колличество - признак: 643
Колличество - целевой признак: 643 

Тестовая база:
Колличество - признак: 643
Колличество - целевой признак: 643 



## Исследуйте модели

Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.

### "Решающее дерево"

In [7]:
best_model_DTC = None
best_result_DTC = 0
best_depth_DTC = 0
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучите модель с заданной глубиной дерева
    model.fit(features_train,target_train) # обучите модель
    predictions_valid = model.predict(features_valid) # получите предсказания модели
    result = accuracy_score(target_valid, predictions_valid) # посчитайте качество модели
    if result > best_result_DTC:
        best_model_DTC = model
        best_result_DTC = result
        best_depth_DTC = depth
        
print("Accuracy лучшей модели:", best_result_DTC, "Глубина дерева:", best_depth_DTC)





Accuracy лучшей модели: 0.7853810264385692 Глубина дерева: 3


### "Случайный лес"

In [8]:
best_model_RFC = None
best_result_RFC = 0
best_est_RFC = 0
best_depth_RFC = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid) 
        result = accuracy_score(target_valid, predictions_valid) 
        if result > best_result_RFC:
            best_model_RFC = model
            best_result_RFC = result
            best_est_RFC = est
            best_depth_RFC = depth

print("Accuracy лучшей модели:", best_result_RFC, "Количество деревьев:", best_est_RFC, "Максимальная глубина:", depth)

Accuracy лучшей модели: 0.8087091757387247 Количество деревьев: 40 Максимальная глубина: 10


### "Линейная регрессия"

In [9]:
model_LR = LogisticRegression() # присваиваем модели имя
model_LR.fit(features_train,target_train) # обучаем
predictions_valid=model_LR.predict(features_valid) # проверяем
print('accuracy:',accuracy_score(target_valid,predictions_valid))

accuracy: 0.7107309486780715


**Промежуточный вывод**

Наиболее удачная модель - "Случайный лес" c Количеством деревьев: 40 и Максимальная глубина: 10


## Проверьте модель на тестовой выборке

Проверьте качество модели на тестовой выборке.

In [10]:
predictions_test = best_model_RFC.predict(features_test) 
result_test = accuracy_score(target_test, predictions_test) 

print('Результат accuracy на тестовой выборке:',result_test)

Результат accuracy на тестовой выборке: 0.7962674961119751


## (бонус) Проверьте модели на адекватность

Дополнительное задание: проверьте модели на вменяемость. Ничего страшного, если не получится: эти данные сложнее тех, с которыми вы работали раньше. В следующем курсе подробнее об этом расскажем.

При создании модели машинного обучения для какого-либо проекта разумно создать baseline модель. Она представляет собой простейшую (dummy) модель, которая всегда предсказывает наиболее часто встречающийся класс. Так мы получим контрольные данные для сравнительной оценки более сложной модели.


In [11]:
# импорт необходимой библиотеки
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent", random_state=0)
dummy_clf.fit(features_train, target_train)
dummy_clf.score(features_train, target_train)



0.6924273858921162

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75


**Итоговый вывод**

* Мы видим, что выюранный "Случайный лес" лучше, чем фиктивный классификатор. **0.80 > 0.69**
* Accuracy лучшей модели на тестовой выборке **~ 0.80, что больше 0.75**, а значит задача выполнена